To start, call the contents of my package, ignore the warnings or errors, this is because somehow we are redefining something by calling to packages that have the same function

In [1]:
using FractionalKrylovMC

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


We create a random problem

In [2]:
# USAGE: problem = create_random_problem(α::ExponentInTime,γ::ExponentInSpace,k::SizeOfVector)
α = 0.9 # using derivative in time ^C_0 D_t^α
γ = 0.8 # using A^γ as the linear operator
# perhaps lower this!
k = 300 # size(u0) = k; size(A) = (k,k)
problem = create_random_problem(α,γ,k)

MittagLefflerProblem{Float64}(300, [0.5415797617907798 0.007448574424917989 … -0.012804403553866523 -0.02155013304435399; 0.007448574424917454 0.47584531891080223 … -0.02323978708140622 -0.01576003731702437; … ; -0.012804403553866382 -0.023239787081405942 … 0.5151243803963529 -0.010176633806810888; -0.021550133044353433 -0.015760037317024193 … -0.010176633806810739 0.46389198478556204], [0.5415797617907798 0.007448574424917989 … -0.012804403553866523 -0.02155013304435399; 0.007448574424917454 0.47584531891080223 … -0.02323978708140622 -0.01576003731702437; … ; -0.012804403553866382 -0.023239787081405942 … 0.5151243803963529 -0.010176633806810888; -0.021550133044353433 -0.015760037317024193 … -0.010176633806810739 0.46389198478556204], [0.09369877043961794, 0.19823725680264281, 0.13476643922525577, 0.8900075975792917, 0.4474145836972563, 0.8228001725726551, 0.5500563635277643, 0.12226622195788306, 0.41502431728205313, 0.606308293886332  …  0.44692808103765647, 0.10112007754377361, 0.248

problem has following fields

In [ ]:
# relevant parameters of the problem
problem.A # matrix
problem.u0 # initial condition
problem.t # time
# we want to compute
# mittleff(α,-A^γ*t^α)*u0

the way I create the problem, I already compute the exact solution

In [ ]:
problem.uT_spectral

we can call the Monte-Carlo solver, which computes the exponential matrix by computing $\exp(-At)$ for all random times $t$ and does not project to any subspace

In [3]:
nsims = Int(1e5)
cutoff = 1e6 # ignoring all times beyond cutoff
MCSol = solve(problem,nsims,cutoff,MCSolver())

MittagLefflerMCSolution{Float64}([0.1161385291640483, 0.03591638625488333, 0.09078153148408165, 0.5871217662439915, 0.3309113002463288, 0.5801671890293563, 0.3770143862958261, 0.12686100906931472, 0.29835599785625905, 0.39932202373723535  …  0.2569133860915448, 0.07289832924252, 0.01803701547952628, 0.087951211289036, 0.70192650844729, 0.1907361056870118, 0.7167252355603713, 0.22218576136826282, 0.3646349173376149, 0.29532673898311274], [1.835556615990041e-6, 2.9277600788924093e-6, 5.997265539885213e-6, 1.0130799875337304e-5, 1.0966673645666167e-5, 1.2642351832009958e-5, 1.4357399960530807e-5, 1.5372632002086625e-5, 1.5454042364485924e-5, 2.333032626147116e-5  …  38594.756580624286, 38915.15024644537, 42392.02148575521, 61645.160422166104, 62905.212626993656, 88634.97494621371, 120901.08215727174, 137382.98680677163, 150506.00516542958, 173746.68787378754], 100000, 1.0e6)

Check relative error and random entries to see whether it is correct, checking with confidence ranges will be done in the future

In [4]:
using Random
@show relative_error(problem.uT_spectral,MCSol.uT);
entries_to_show = randperm(k)[1:5]
@show problem.uT_spectral[entries_to_show];
@show MCSol.uT[entries_to_show];

relative_error(problem.uT_spectral, MCSol.uT) = 0.04405151545783162
problem.uT_spectral[entries_to_show] = [0.4948437169610488, 0.18954693192741814, 0.4209849095435812, 0.09666831905705973, 0.36221753650400645]
MCSol.uT[entries_to_show] = [0.5137729592988232, 0.1884506304314795, 0.46494053242545663, 0.087951211289036, 0.3779317406972008]


can also use Krylov!

In [5]:
KrySubspaceDim = 5
MCKrySol = solve(problem,nsims,cutoff,KrySubspaceDim,MCSolverExpokit())

MittagLefflerMCSolution{Float64}([0.11636792768055922, 0.03590590232145222, 0.09095050889481589, 0.5877801084548403, 0.3312206673144261, 0.5808196195255234, 0.3774191449106153, 0.12705843474498177, 0.2987390918440006, 0.399895148964329  …  0.25722577350422654, 0.07298003390713695, 0.018049700787571336, 0.08802300657120808, 0.7026595843706074, 0.19096146424429045, 0.7175306864704117, 0.22248998464868872, 0.36509778629551143, 0.29578879059167656], [2.2923520499773145e-6, 3.799145958859502e-6, 1.5132093552727634e-5, 1.6060090715816866e-5, 1.7576099950020373e-5, 1.9807230255466673e-5, 2.11699004618462e-5, 2.614284110197647e-5, 2.758549532932784e-5, 2.8661746962706307e-5  …  23523.68249820971, 24975.751542023347, 26856.316512089554, 27225.547935359755, 28895.5865803514, 31755.969661765645, 38945.8723638231, 48549.76383027059, 55477.94722272533, 84481.47328431379], 100000, 1.0e6)

In [6]:
@show relative_error(problem.uT_spectral,MCSol.uT);
entries_to_show = randperm(k)[1:5]
@show problem.uT_spectral[entries_to_show];
@show MCKrySol.uT[entries_to_show];

relative_error(problem.uT_spectral, MCSol.uT) = 0.04405151545783162
problem.uT_spectral[entries_to_show] = [0.32903371631807354, 0.466519351346999, 0.39266344200827225, 0.42292447860218824, 0.3700733660968992]
MCKrySol.uT[entries_to_show] = [0.3435055952725601, 0.48435786832338157, 0.40558008261410644, 0.4253704811065888, 0.3735129089014563]
